# Setup

In [ ]:
# Imports
import collections
import os
import os.path as osp

import joblib
import numpy as np
import gym
import pandas as pd
import matplotlib.cm as cm
import matplotlib.pyplot as plt

import pirl
from pirl import config
from pirl.experiments import experiments

from analysis import common

In [ ]:
# Config
experiments = [
    'few-jungle-9x9-Liquid-20180521_041300-ddc8132bb2a30ef149e3b567d84ccacd168e045c',
    'few-jungle-9x9-Water-20180521_041259-ddc8132bb2a30ef149e3b567d84ccacd168e045c',
    'few-jungle-9x9-Soda-20180521_041300-ddc8132bb2a30ef149e3b567d84ccacd168e045c',
]
experiments_dir = [osp.join('..', 'data', 'aws', 'experiments', x) for x in experiments]
figs_dir = osp.join('../../population-irl-paper', 'figs')

# Value difference

In [ ]:
algo_pattern = '(.*)'
env_pattern = '.*-(.*)-v0'
dfs = [common.load_value(x, algo_pattern, env_pattern) for x in experiments_dir]
df = pd.concat(dfs)
values = common.aggregate_value(df, 1)

In [ ]:
common.plot_ci(values)

# Policy rollout

In [ ]:
def expert_cached_value(rl, env_name, pol_discount=0.99, eval_discount=1.00, seed=1234, episodes=100):
    '''Rollout a cached expert policy for episodes.
       WARNING: This will be slow or just break if policy is not in cache!'''
    gen_policy, _sample, compute_value = config.RL_ALGORITHMS[rl]
    policy, value = experiments._train_policy(rl, pol_discount, env_name, seed, None)
    vmean, vse = value
    print('Cached value: {:.3f} +/- {:.3f}'.format(vmean, 1.96 * vse))
    
    env = gym.make(env_name)
    rmean, rse = compute_value(env, policy, eval_discount, num_episodes=episodes, seed=seed)
    print('Rollout value: {:.3f} +/- {:.3f}'.format(rmean, 1.96 * rse))
    return (vmean, vse), (rmean, rse)

def _policy_value(results_dir, rl, env_name, pol_discount, eval_discount, episodes, seed):
    _gen_policy, _sample, compute_value = config.RL_ALGORITHMS[rl]
    fname = osp.join(results_dir, 'policy.pkl')
    print('Loading policy from ', fname)
    policy = joblib.load(fname)
    env = gym.make(env_name)
    
    mean, se = compute_value(env, policy, eval_discount, num_episodes=episodes, seed=seed)
    print('Rollout value: {:.3f} +/- {:.3f}'.format(mean, 1.96 * se))
    return mean, se

def expert_value(experiment_dir, rl, env_name, pol_discount=0.99, eval_discount=1.00, episodes=100, seed=1234):
    results_dir = osp.join(experiment_dir, 'expert', env_name, rl)
    return _policy_value(results_dir, rl, env_name, pol_discount, eval_discount, episodes, seed)

def irl_eval_value(experiment_dir, irl_name, num_traj, rl, env_name, pol_discount=0.99, eval_discount=1.00, episodes=100, seed=1234):
    results_dir = osp.join(experiment_dir, 'eval', env_name, 
                           '{}:{}:{}'.format(irl_name, num_traj, num_traj), rl)
    return _policy_value(results_dir, rl, env_name, pol_discount, eval_discount, episodes, seed)
    
def irl_value(experiment_dir, irl_name, env_name, num_traj, eval_discount=1.00, episodes=100):
    _irl_algo, _reward_wrapper, compute_value = experiments.make_irl_algo(irl_name)
    irl_dir = osp.join(experiment_dir, 'irl', irl_name)
    if not os.path.exists(irl_dir):
        raise FileNotFoundError("No result directory {}".format(irl_dir))
    
    pop_fname = osp.join(irl_dir, str(num_traj), 'policies.pkl')
    sin_fname = osp.join(irl_dir, env_name, str(num_traj), 'policy.pkl')
    if os.path.exists(pop_fname):
        policies = joblib.load(pop_fname)
        print(policies.keys())
        policy = policies[env_name]
    elif os.path.exists(sin_fname):
        policy = joblib.load(sin_fname)
    else:
        raise FileNotFoundError("Neither {} or {} exists".format(pop_fname, sin_fname))
    
    env = gym.make(env_name)
    mean, se = compute_value(env, policy, discount=eval_discount, num_episodes=episodes)
    print('Rollout value: {} +/- {}'.format(mean, 1.96 * se))

In [ ]:
expert_cached_value('ppo_cts', 'Reacher-v2', episodes=100)

In [ ]:
expert_value(experiment_dir, 'ppo_cts', 'Reacher-v2', episodes=500, seed=1234)

In [ ]:
irl_value(experiment_dir, 'airl', 'Reacher-v2', 1000, episodes=100)

# Visualizing rewards (gridworld only)

In [ ]:
def show_heatmaps(irl_algo, kind='inline', out_dir=None, shape=(9,9), **kwargs):
    data = pd.read_pickle(osp.join(experiment_dir, 'results.pkl'))
    rewards = data['rewards'][irl_algo]
    if kind in ['inline', 'pdf']:
        figs = myplots.gridworld_heatmap(rewards, shape)
        if out_dir is None:
            for fig in figs:
                display(fig[1])
        else:
            myplots.save_figs(figs, out_dir)
    elif kind == 'movie':
        myplots.gridworld_heatmap_movie(out_dir, rewards, shape)
    else:
        assert False

In [ ]:
irl_algos = ['mce', 'mcec', 'mcep_reg1e0', 'mcep_reg1e-1', 'mcep_reg1e-2']
for irl in irl_algos:
    show_heatmaps(irl, kind='pdf', out_dir='figs/few-jungle/' + irl, shape=(9,9))
    #show_heatmaps(irl, kind='movie', out_dir='figs/jungle/movies/' + irl)
    #show_heatmaps(irl, kind='movie', out_dir='figs/jungle/' + irl)

# Jungle experiments

In [ ]:
jungle_types = collections.OrderedDict([
    ('A', 'Soda'), 
    ('B', 'Water'), 
    ('A+B', 'Liquid')
])
jungle_envs = collections.OrderedDict([
    (k, 'pirl/GridWorld-Jungle-9x9-{}-v0'.format(v))
    for k, v in jungle_types.items()
])
default_algos = collections.OrderedDict([
    ('mce', 'Single'),
    ('mcec', 'Joint Training'),
    ('mcep reg1e-1', 'Multi-Task'),
    ('value iteration', 'Oracle'),
])

## Ground truth

In [ ]:
with plt.style.context([common.style('default'), common.style('twocol')]):
    fig = common.gridworld_ground_truth(jungle_envs, (9,9))
    fig.savefig(osp.join(figs_dir, 'jungle', 'gt.pdf'))
    plt.close(fig)

# Cartoon

In [ ]:
with plt.style.context([common.style('default'), common.style('onecol')]):
    fig = common.gridworld_cartoon((9,9))
    fig.savefig(osp.join(figs_dir, 'jungle', 'cartoon.pdf'))
    #plt.close(fig)

## Figures

In [ ]:
jungle_val = df.stack().unstack('seed').max(axis=1).unstack(-1)
jungle_val = jungle_val.rename(index={v: k for k, v in jungle_types.items()}, level=0)
jungle_val = jungle_val.xs((1000, 'value_iteration'), level=('n', 'eval'))
baseline_comparison = jungle_val.rename(columns=default_algos).loc[:, default_algos.values()]
with plt.style.context([common.style('default'), common.style('twocol')]):
    fig = common.value_bar_chart_by_env(baseline_comparison, envs=jungle_types.keys(), relative='Oracle')
    fig.savefig(osp.join(figs_dir, 'jungle', 'baseline_comparison.pdf'), bbox_inches='tight')

In [ ]:
reg_algorithms = collections.OrderedDict([
    #('mcep reg1e-4', '$\lambda = 10^{-4}$'),
    #('mcep reg1e-3', '$\lambda = 10^{-3}$'),
    ('mcep reg1e-2', '$\lambda = 10^{-2}$'),
    ('mcep reg1e-1', '$\lambda = 10^{-1}$'),
    ('mcep reg1e0', '$\lambda = 10^0$'),
    ('value iteration', 'Oracle'),
])
jungle_val = values.rename(index={v: k for k, v in jungle_types.items()}, level=0)
jungle_val = jungle_val.xs((1000, 'value_iteration'), level=('n', 'eval'))
reg_comparison = jungle_val.rename(columns=reg_algorithms).loc[:, reg_algorithms.values()]
with plt.style.context([common.style('default'), common.style('twocol')]):
    fig = common.value_bar_chart_by_env(reg_comparison, envs=jungle_types.keys(), 
                                         relative='Oracle', cmap=cm.get_cmap('Accent'))
    fig.savefig(osp.join(figs_dir, 'jungle', 'reg_comparison.pdf'), bbox_inches='tight')

## Tables

In [ ]:
ci = common.plot_ci(values, dp=1)
#ci = ci.str.replace('nan +/- nan', '---')
ci = ci.rename(index={v: k for k, v in jungle_types.items()}, level=0)
ci = ci.rename(columns=default_algos).loc[:, default_algos.values()]
ci.columns.name = 'algo'
#ci = ci.unstack('env')
#ci.columns = ci.columns.reorder_levels(['env', 'algo'])
#envs = ci.columns.levels[0]
#ci = pd.concat([ci.loc[:, [x]] for x in envs], axis=1)
ci = ci.xs((1000, 'value_iteration'), level=('n', 'eval'))
print(ci.to_latex())

In [ ]:
table_values = values.rename(columns=default_algos).loc[:, default_algos.values()]
print(common.value_latex_table(table_values, envs=['A', 'B', 'A+B'])), relative='Oracle'))

# Ad-hoc experiments

In [ ]:
fname = osp.join(experiment_dir, 'results.pkl')
data = pd.read_pickle(fname)

In [ ]:
env_name = 'pirl/GridWorld-Jungle-9x9-Soda-v0'
algo = 'mce'
fig, axs = plt.subplots(1, 2, figsize=(13, 6))
# zeroshot = data['rewards'][algo][1000][0][env_name]
# oneshot = data['rewards'][algo][1000][1][env_name]
# myplots._gridworld_heatmap(oneshot - zeroshot, (9,9), fmt='.2f', ax=axs[0])
fiveshot = data['rewards'][algo][1000][5][env_name]
hundredshot = data['rewards'][algo][1000][100][env_name]
myplots._gridworld_heatmap(hundredshot - zeroshot, (9,9), fmt='.2f', ax=axs[1])

env = gym.make(env_name)
gt = env.unwrapped.reward
env.close()
r = data['rewards'][algo][1000]
reward_delta = {k: np.linalg.norm(v[env_name] - np.mean(v[env_name]) - gt + np.mean(gt))
                for k, v in r.items()}
reward_delta